# Hybrid system

- Coger subsets aleatorios
- Ver si usar item based collab porque tiene mas escalabilidad, user based necesita calcular la similitud de todos los pares de usuarios

## Data loading

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
RANDOM_STATE = 123

In [2]:
#df_users = pd.read_csv('data/train_triplets.txt', sep='\t', header=None, names=['user_id', 'song_id', 'play_count'])
df_users_raw = pd.read_csv('data/Million Song Dataset kaggle/User Listening History.csv')

In [3]:
df_music = pd.read_csv('data/Million Song Dataset kaggle/Music Info.csv')

In [4]:
df_users_raw.dtypes

track_id     object
user_id      object
playcount     int64
dtype: object

In [5]:
df_users_raw.shape

(9711301, 3)

In [6]:
num_distinct_users = df_users_raw['user_id'].nunique()
num_distinct_users

962037

We only take into account users that have listened to at least 10 songs

In [7]:
df_user_counts = df_users_raw['user_id'].value_counts().reset_index()
df_user_counts.columns = ['user_id', 'count']
df_user_counts

,user_id,count
0,ec6dfcf19485cb011e0b22637075037aae34cf26,784
1,4e11f45d732f4861772b2906f81a7d384552ad12,384
2,726da71c2c2ea119119a7957517fccd028d1be76,376
3,113255a012b2affeab62607563d03fbdf31b08e7,367
4,7adec7f006cb09482d36609d205293d8b61f030e,366
...,...,...
962032,19e70c61de7bbb0ca3fc39c81a7e6b7eede69aa1,1
962033,7154895e6b692062033eecb3fe59ad240bb02811,1
962034,5d563bc9f80ddb3bce489c84b453409a88bd37e9,1
962035,779074ba35bd5f14913635819233a01dc5ecc9de,1


In [8]:
df_users = df_users_raw[df_users_raw['user_id'].isin(df_user_counts[df_user_counts['count'] > 5]['user_id'])].reset_index(drop=True)
#df_users = df_users_raw
df_users.shape

(8332242, 3)

In [9]:
data_df = df_users_raw.groupby('track_id').size().reset_index(name='count')

In [10]:
data_df

,track_id,count
0,TRAAAED128E0783FAB,161
1,TRAAAHD128F42635A5,2
2,TRAAAQO12903CD8E1C,1
3,TRAABJS128F9325C99,945
4,TRAABXA128F9326351,18
...,...,...
30454,TRZZYMU128E0792400,1140
30455,TRZZZCL128F428BB80,19
30456,TRZZZCN128F9317A03,15
30457,TRZZZHL128F9329CFB,116


In [11]:
df_music.head()

,track_id,name,artist,spotify_preview_url,spotify_id,tags,genre,year,duration_ms,danceability,...,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,time_signature
0,TRIOREW128F424EAF0,Mr. Brightside,The Killers,https://p.scdn.co/mp3-preview/4d26180e6961fd46...,09ZQ5TmUG8TSL56n0knqrj,"rock, alternative, indie, alternative_rock, in...",NaN,2004,222200,0.355,...,1,-4.360,1,0.0746,0.001190,0.000000,0.0971,0.240,148.114,4
1,TRRIVDJ128F429B0E8,Wonderwall,Oasis,https://p.scdn.co/mp3-preview/d012e536916c927b...,06UfBBDISthj1ZJAtX4xjj,"rock, alternative, indie, pop, alternative_roc...",NaN,2006,258613,0.409,...,2,-4.373,1,0.0336,0.000807,0.000000,0.2070,0.651,174.426,4
2,TROUVHL128F426C441,Come as You Are,Nirvana,https://p.scdn.co/mp3-preview/a1c11bb1cb231031...,0keNu0t0tqsWtExGM3nT1D,"rock, alternative, alternative_rock, 90s, grunge",RnB,1991,218920,0.508,...,4,-5.783,0,0.0400,0.000175,0.000459,0.0878,0.543,120.012,4
3,TRUEIND128F93038C4,Take Me Out,Franz Ferdinand,https://p.scdn.co/mp3-preview/399c401370438be4...,0ancVQ9wEcHVd0RrGICTE4,"rock, alternative, indie, alternative_rock, in...",NaN,2004,237026,0.279,...,9,-8.851,1,0.0371,0.000389,0.000655,0.1330,0.490,104.560,4
4,TRLNZBD128F935E4D8,Creep,Radiohead,https://p.scdn.co/mp3-preview/e7eb60e9466bc3a2...,01QoK9DA7VTeTSE3MNzp4I,"rock, alternative, indie, alternative_rock, in...",RnB,2008,238640,0.515,...,7,-9.935,1,0.0369,0.010200,0.000141,0.1290,0.104,91.841,4


In [12]:
df_music_info = df_music[['track_id', 'name', 'artist', 'energy']]
df_music_info.head()

,track_id,name,artist,energy
0,TRIOREW128F424EAF0,Mr. Brightside,The Killers,0.918
1,TRRIVDJ128F429B0E8,Wonderwall,Oasis,0.892
2,TROUVHL128F426C441,Come as You Are,Nirvana,0.826
3,TRUEIND128F93038C4,Take Me Out,Franz Ferdinand,0.664
4,TRLNZBD128F935E4D8,Creep,Radiohead,0.430


In [13]:
df_music_count = df_music_info.merge(
    df_users.groupby('track_id').size().reset_index(name='playcount'),
    on='track_id',
    how='left'
)
df_music_count = df_music_count.fillna(0)
print(df_music_count.shape)
df_music_count.head()

(50683, 5)


,track_id,name,artist,energy,playcount
0,TRIOREW128F424EAF0,Mr. Brightside,The Killers,0.918,32.0
1,TRRIVDJ128F429B0E8,Wonderwall,Oasis,0.892,283.0
2,TROUVHL128F426C441,Come as You Are,Nirvana,0.826,0.0
3,TRUEIND128F93038C4,Take Me Out,Franz Ferdinand,0.664,17.0
4,TRLNZBD128F935E4D8,Creep,Radiohead,0.430,0.0


We load the asigned cluster for each song previously calculated

In [14]:
id_to_cluster = pd.read_csv('data/track_clusters.csv', index_col=0).iloc[:, 0]

In [15]:
id_to_cluster

track_id
TRIOREW128F424EAF0    4
TRRIVDJ128F429B0E8    4
TROUVHL128F426C441    3
TRUEIND128F93038C4    2
TRLNZBD128F935E4D8    2
                     ..
TRQYCFV128F9322F50    3
TRHQCSH128F42724B7    4
TRZRODK128F92D68D7    0
TRGLMEM128F9322F63    3
TRIPFKO128F42383FE    2
Name: 0, Length: 50683, dtype: int64

## Interaction Matrix

In [16]:
df_users_agg = df_users.groupby('user_id')['playcount'].agg(
    #total_playcount='sum',
    max_playcount='max'
).reset_index()
df_users_agg = df_users_agg.rename(columns={'playcount': 'max_playcount'})

In [17]:
df_users_rating = df_users.merge(df_users_agg, on='user_id')

We create 2 interaction matrix
- Binary version: 1 indicates a user has interacted with the song and 0 indicates he has not
- Weighted version: Rating in range (0,1], calculated as (user playcount) / (playcount of user's most listened song). Gives more rating to those more listened by a user

In [18]:
df_users_rating['rating'] = df_users_rating['playcount'] / df_users_rating['max_playcount']

In [19]:
df_users_agg[df_users_agg['user_id'] == 'b80344d063b5ccb3212f76538f3d9e43d87dca9e']

,user_id,max_playcount
333783,b80344d063b5ccb3212f76538f3d9e43d87dca9e,2


For the interaction matrix, we shall use a sparse matrix, due to users only listening to a small subset of the songs

In [20]:
from scipy.sparse import csr_matrix, csc_matrix

We codify each user and song to a unique integer ID

In [21]:
user_codes, user_uniques = pd.factorize(df_users['user_id'])
track_codes, track_uniques = pd.factorize(df_users['track_id'])

In [22]:
interaction_matrix_user_item = csr_matrix((df_users_rating['playcount'], (user_codes, track_codes)), shape=(len(user_uniques), len(track_uniques)))

## Matrix Factorization: Alternating Least Squares (ALS)

In [23]:
from implicit.als import AlternatingLeastSquares

In [106]:
class ALSRecommender:
    def __init__(self, interaction_matrix, track_uniques, df_music_info, als_model=None):
        self.interaction_matrix = interaction_matrix
        self.track_uniques = track_uniques
        self.df_music_info = df_music_info

        if als_model is None:
            self.als_model = AlternatingLeastSquares(factors=50, regularization=0.1, iterations=20, num_threads=-1, random_state=RANDOM_STATE)
            self.als_model.fit(self.interaction_matrix)
        else:
            self.als_model = als_model

        self.user_index = None
        self.recommendations = None # List of tuples (track_id, energy, similarity, has been recommended)

    def make_recommendations(self, user_index, n=100):
        self.user_index = user_index

        user_items = self.interaction_matrix.tocsr()[user_index]

        top_n_recommendations_indexes, top_n_recommendations_scores = self.als_model.recommend(user_index, user_items, N=n, filter_already_liked_items=True)

        track_ids = self.track_uniques[top_n_recommendations_indexes].tolist()
        
        df_filtered = self.df_music_info.set_index('track_id').loc[track_ids][['energy']].reset_index()

        self.recommendations = [(track_id, energy, similarity, False) for (track_id, energy), similarity in zip(df_filtered.itertuples(index=False, name=None), top_n_recommendations_scores)]

        return self.recommendations

    
    def recommend_song(self, energy, energy_margin=0.05):
        if self.recommendations is None:
            raise ValueError("No recommendations available. Please call make_recommendations first.")
        
        closest_track_index = None
        distance_to_energy = float('inf')

        for i, (track_id, track_energy, similarity, has_been_recommended) in enumerate(self.recommendations):
            distance = abs(track_energy - energy)

            if not has_been_recommended and distance <= energy_margin:
                self.recommendations[i] = (track_id, track_energy, similarity, True)
                return (track_id, track_energy)
            
            if not has_been_recommended and distance < distance_to_energy:
                closest_track_index = i
                distance_to_energy = distance
        
        if closest_track_index is not None:
            track_id, track_energy, _, _= self.recommendations[closest_track_index]
            self.recommendations[closest_track_index] = (track_id, track_energy, similarity, True)
            return (track_id, track_energy)

        raise ValueError("All recommendations have already been recommended")


    def get_recommendations(self):
        if self.recommendations is None:
            raise ValueError("No recommendations available. Please call make_recommendations first.")
        return self.recommendations


    def get_recommendations_ids(self):
        if self.recommendations is None:
            raise ValueError("No recommendations available. Please call make_recommendations first.")
        return [track_id for track_id, _, _, _ in self.recommendations]
    
    def get_recommendations_info(self):
        track_ids = [track_id for track_id, _, _, _ in self.recommendations]
        df_ordered = self.df_music_info.set_index('track_id').loc[track_ids].reset_index()
        return df_ordered

## Kmeans content-based filtering

In [107]:
class KmeansContentBasedRecommender: #For user history
    def __init__(self, id_to_cluster):
        self.id_to_cluster = id_to_cluster
        self.recommendations = None
    
    def make_cluster_recommendation(self, user_history):
        clusters = self.id_to_cluster[user_history]
        cluster_counts = clusters.value_counts()
        self.recommended_cluster = cluster_counts / len(clusters)
        return self.recommended_cluster

    def get_recommended_cluster(self):
        if self.recommended_cluster is None:
            raise ValueError("No cluster recommendation available. Please call make_cluster_recommendation first.")
        return self.recommendations
    

## Hybrid recommender

In [ ]:
class HybridRecommender:
    def __init__(self, interaction_matrix_user_item, track_uniques, df_music_info, df_users, id_to_cluster, als_recommender = None, content_based_recommender = None):
        if als_recommender is not None:
            self.collaborative_als_recommender = als_recommender
        else:
            self.collaborative_als_recommender = ALSRecommender(interaction_matrix_user_item, track_uniques, df_music_info)
        
        if content_based_recommender is not None:
            self.content_based_recommender = content_based_recommender  
        else:
            self.content_based_recommender = KmeansContentBasedRecommender(id_to_cluster)

        self.df_music_info = df_music_info
        self.df_users = df_users
        self.id_to_cluster = id_to_cluster
        self.recommendations = None # List of tuples (track_id, energy, similarity, has been recommended)

    
    def make_recommendations(self, user_index, n=100):

        user_id = df_users['user_id'].unique()[user_index]
        user_history = self.df_users[self.df_users['user_id'] == user_id]['track_id']
        collaborative_recomendations = self.collaborative_als_recommender.make_recommendations(user_index, n)
        content_based_cluster_recommendation = self.content_based_recommender.make_cluster_recommendation(user_history)
        self.recommendations = []
        
        #We will apply a penalization to the collaborative filtering recommendation based on the user cluster preferences obtained by the content-based recommendation
        for track_id, energy, similarity, has_been_recommended in collaborative_recomendations:
            multiplier = 0.01 #Default multiplier. Used if the song's cluster is not in the user's cluster preferences (content-based recommendation)
            song_cluster = self.id_to_cluster[track_id]
            if song_cluster in content_based_cluster_recommendation.index:
                multiplier = content_based_cluster_recommendation[song_cluster]
            
            #print(track_id, song_cluster, multiplier)

            self.recommendations.append((track_id, energy, similarity * multiplier, has_been_recommended))
        self.recommendations = sorted(self.recommendations, key=lambda x: x[2], reverse=True)  # Sort new similarity


    def make_recommendations_only_collaborative(self, user_index, n=100):
        self.recommendations = self.collaborative_als_recommender.make_recommendations(user_index, n)
    
    def recommend_song(self, energy, energy_margin=0.05):
        return self.als_recommender.recommend_song(energy, energy_margin)
    
    def get_recommendations(self):
        if self.recommendations is None:
            raise ValueError("No recommendations available. Please call make_recommendations first.")
        return self.recommendations
    
    def get_recommendations_ids(self):
        if self.recommendations is None:
            raise ValueError("No recommendations available. Please call make_recommendations first.")
        return [track_id for track_id, _, _, _ in self.recommendations]
    
    def get_recommendations_info(self):
        if self.recommendations is None:
            raise ValueError("No recommendations available. Please call make_recommendations first.")
        track_ids = [track_id for track_id, _, _, _ in self.recommendations]
        df_ordered = self.df_music_info.set_index('track_id').loc[track_ids].reset_index()
        return df_ordered

### Example

In [27]:
user_index = 0 #User for wich recommendations will be made
# user_listened_songs = df_users[df_users['user_id'] == user_uniques[user_index]].track_id
# df_music_info[df_music_info['track_id'].isin(user_listened_songs)]

In [28]:
als_model = AlternatingLeastSquares(factors=50, regularization=0.1, iterations=20, num_threads=-1, random_state=RANDOM_STATE)
als_model.fit(interaction_matrix_user_item)

c:\Users\Dani\.conda\envs\Master-IA\lib\site-packages\implicit\cpu\als.py:95: RuntimeWarning: OpenBLAS is configured to use 12 threads. It is highly recommended to disable its internal threadpool by setting the environment variable 'OPENBLAS_NUM_THREADS=1' or by calling 'threadpoolctl.threadpool_limits(1, "blas")'. Having OpenBLAS use a threadpool can lead to severe performance issues here.
  check_blas_config()


  0%|          | 0/20 [00:00<?, ?it/s]

In [109]:
als_recommender = ALSRecommender(interaction_matrix_user_item, track_uniques, df_music_info, als_model=als_model)

In [171]:
hybrid_recommender = HybridRecommender(interaction_matrix_user_item, track_uniques, df_music_info, df_users, id_to_cluster, als_recommender=als_recommender)
hybrid_recommender.make_recommendations_only_collaborative(user_index, n=10)
hybrid_recommender.get_recommendations_info().head(20)

,track_id,name,artist,energy
0,TRUJOHU128F424E6A6,White Winter Hymnal,Fleet Foxes,0.4970
1,TRQEMRN128F933B4B2,Sun It Rises,Fleet Foxes,0.2750
2,TRGEIDA128F933B4B8,Tiger Mountain Peasant Song,Fleet Foxes,0.1460
3,TRVRIBZ128F426BF71,Your Protector,Fleet Foxes,0.5370
4,TRLXFKL128F426BF75,Oliver James,Fleet Foxes,0.0635
5,TRKABMK128F426BF72,Meadowlarks,Fleet Foxes,0.1660
6,TRFTUIW128E0784B9F,Bubble Toes,Jack Johnson,0.5320
7,TRDJRES128F933B4BA,Quiet Houses,Fleet Foxes,0.6680
8,TRXWAZC128F9314B3E,Heartbreak Warfare,John Mayer,0.5720
9,TRCQYJE128E078FA49,The News,Jack Johnson,0.0818


In [172]:
hybrid_recommender.make_recommendations(user_index, n=10)
hybrid_recommender.get_recommendations_info().head(20)

,track_id,name,artist,energy
0,TRUJOHU128F424E6A6,White Winter Hymnal,Fleet Foxes,0.4970
1,TRFTUIW128E0784B9F,Bubble Toes,Jack Johnson,0.5320
2,TRXWAZC128F9314B3E,Heartbreak Warfare,John Mayer,0.5720
3,TRQEMRN128F933B4B2,Sun It Rises,Fleet Foxes,0.2750
4,TRGEIDA128F933B4B8,Tiger Mountain Peasant Song,Fleet Foxes,0.1460
5,TRLXFKL128F426BF75,Oliver James,Fleet Foxes,0.0635
6,TRKABMK128F426BF72,Meadowlarks,Fleet Foxes,0.1660
7,TRCQYJE128E078FA49,The News,Jack Johnson,0.0818
8,TRVRIBZ128F426BF71,Your Protector,Fleet Foxes,0.5370
9,TRDJRES128F933B4BA,Quiet Houses,Fleet Foxes,0.6680
